In [1]:
# importing the required modules
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from time import perf_counter 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten
import cv2
import numpy as np

In [2]:
# Defining the paths to the data folders
train_dir = '../data/train'
test_dir = '../data/test'
valid_dir = '../data/valid'

In [3]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load the train data
train_data = datagen.flow_from_directory(train_dir, 
                                         target_size=(128, 128), # Adjust to your image size
                                         class_mode='categorical',)

# Load the validation data
valid_data = datagen.flow_from_directory(valid_dir, 
                                         target_size=(128, 128), 
                                         class_mode='categorical')

# Load the test data
test_data = datagen.flow_from_directory(test_dir, 
                                        target_size=(128, 128), 
                                        class_mode='categorical', 
                                        shuffle=False)  # No need to shuffle test data

In [5]:
from keras.layers import Input, BatchNormalization, Dropout

In [6]:
model = Sequential([
    Input(shape=(128,128,3)),

    Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=2),
    BatchNormalization(axis=1),
    Dropout(0.25),

    Conv2D(filters=32, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=2),

    Flatten(),
    Dense(units=32,activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.5),
    Dense(units=13,activation='softmax')
])

In [ ]:
model.summary()

In [9]:
from tensorflow.keras.optimizers import Adam

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"],
              )

In [11]:
import keras

In [ ]:
epochs = 100
checkpoint_filepath = '../models/checkpoint.model.keras'

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

accuracy_metrics = model.fit(
    x=train_data,
    epochs=epochs,
    validation_data=valid_data,
    batch_size=20,
    callbacks=[model_checkpoint_callback])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(accuracy_metrics.history['accuracy'],label='train_accuracy')
plt.plot(accuracy_metrics.history['val_accuracy'],label='val_acuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

In [ ]:
print(f"At the end of the {epochs}th epoch the validation accuracy has reached {'{:.4f}'.format(accuracy_metrics.history['val_accuracy'][-1])}")

In [ ]:
from matplotlib import pyplot as plt
plt.plot(accuracy_metrics.history['loss'],label='train_loss')
plt.plot(accuracy_metrics.history['val_loss'],label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [18]:
print(f"At the end of the {epochs}th epoch the validation loss has decreased to {'{:.4f}'.format(accuracy_metrics.history['val_loss'][-1])}")

At the end of the 50th epoch the validation loss has decreased to 2.9393


In [ ]:
model.save('models/baseCNN_reluAdam_second.h5')

In [14]:
model.save('models/baseCNN_reluAdam_second.keras')

In [ ]:
valid_pred1 = model.predict(valid_data)

In [ ]:
valid_pred1.shape

In [22]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [23]:
valid_pred_labels1 = valid_pred1.argmax(axis=1)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(valid_data.labels, valid_pred_labels1)).plot()

In [1]:
# Import packages
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

tf.keras.backend.set_floatx('float64')

# Define random seed for whole notebook
RSEED=42

In [ ]:
# Load the saved model
with tf.device('/cpu:0'):
    new_large_model = tf.keras.models.load_model(r'../models/baseCNN_reluAdam.keras')

# Check its architecture
new_large_model.summary()

In [ ]:
valid_pred = new_large_model.predict(valid_data)

In [ ]:
valid_pred.shape

In [ ]:
valid_data.labels

In [26]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [22]:
valid_pred_labels = valid_pred.argmax(axis=1)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(valid_data.labels, valid_pred_labels)).plot()

In [ ]:
valid_pred[0]

In [ ]:
valid_pred[2].argmax()

In [ ]:
valid_pred[0].argmax()

In [ ]:
# Load the saved model
with tf.device('/cpu:0'):
    new_large_model1 = tf.keras.models.load_model('../models/canhealth_VGG16.keras')

# Check its architecture
new_large_model1.summary()

In [ ]:
pred_valid_vgg = new_large_model1.predict(valid_data)

In [35]:
valid_pred_labels_vgg = pred_valid_vgg.argmax(axis=1)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(valid_data.labels, valid_pred_labels_vgg)).plot()